## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-15-18-58-41 +0000,nypost,"Catholics, conservatives cheer ouster of ex-Mi...",https://nypost.com/2026/02/15/us-news/conserva...
1,2026-02-15-18-48-38 +0000,nypost,Schumer says there are 3 sticking points in DH...,https://nypost.com/2026/02/15/us-news/chuck-sc...
2,2026-02-15-18-46-28 +0000,nyt,‘Wuthering Heights’ Gallops Toward $82 Million...,https://www.nytimes.com/2026/02/15/business/wu...
3,2026-02-15-18-46-12 +0000,startribune,"Live: More than 1,000 federal agents have alre...",https://www.startribune.com/ice-raids-minnesot...
4,2026-02-15-18-42-42 +0000,bbc,"Eleven killed in Israeli strikes on Gaza, resc...",https://www.bbc.com/news/articles/c8r15eyvk2do...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2461/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
122,trump,13
83,guthrie,9
18,ice,9
238,new,8
82,nancy,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
9,2026-02-15-18-21-40 +0000,nypost,DNA found on glove located near Nancy Guthrie’...,https://nypost.com/2026/02/15/us-news/nancy-gu...,42
53,2026-02-15-13-39-00 +0000,wsj,While the main focus of the Munich Security Co...,https://www.wsj.com/politics/policy/europeans-...,38
90,2026-02-15-07-56-40 +0000,nypost,Retired FBI agent urges rapid DNA testing in N...,https://nypost.com/2026/02/15/us-news/retired-...,38
116,2026-02-15-02-00-00 +0000,wsj,"Inflation is easing, jobs are holding up, and ...",https://www.wsj.com/economy/federal-reserve-so...,36
147,2026-02-14-20-48-00 +0000,bbc,No arrests made after police search residence ...,https://www.bbc.com/news/articles/cly8j19rql7o...,34


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
9,42,2026-02-15-18-21-40 +0000,nypost,DNA found on glove located near Nancy Guthrie’...,https://nypost.com/2026/02/15/us-news/nancy-gu...
53,38,2026-02-15-13-39-00 +0000,wsj,While the main focus of the Munich Security Co...,https://www.wsj.com/politics/policy/europeans-...
116,36,2026-02-15-02-00-00 +0000,wsj,"Inflation is easing, jobs are holding up, and ...",https://www.wsj.com/economy/federal-reserve-so...
87,33,2026-02-15-09-00-00 +0000,cbc,The art of no deal: A Greenlandic carver’s ven...,https://www.cbc.ca/news/world/greenland-trump-...
62,29,2026-02-15-13-00-00 +0000,nypost,Far-left activists radicalizing high school ki...,https://nypost.com/2026/02/15/us-news/far-left...
151,24,2026-02-14-20-10-23 +0000,latimes,Factory-built housing hasn't taken off in Cali...,https://www.latimes.com/california/story/2026-...
113,23,2026-02-15-02-04-02 +0000,bbc,Russia killed opposition leader Alexei Navalny...,https://www.bbc.com/news/articles/cwyk4lz4e3eo...
51,23,2026-02-15-14-00-00 +0000,cbc,Gisèle Pelicot has been called a lot of things...,https://www.cbc.ca/radio/sunday/gisele-pelicot...
109,23,2026-02-15-03-15-20 +0000,nypost,Roger Goodell thinks his reckless NFL antics w...,https://nypost.com/2026/02/14/business/roger-g...
43,22,2026-02-15-15-07-40 +0000,bbc,Iran ready to discuss compromises to reach nuc...,https://www.bbc.com/news/articles/clyz4y3zwz5o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
